# Topics in English Quran

In [1]:
import glob

from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
# from nltk.cluster import KMeansClusterer, euclidean_distance
# from gensim import corpora, models, utils
from numpy import array
from gensim import corpora, models


In [7]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

### Downloaded http://www.clearquran.com/

In [42]:
data_folder = '../data/quran-verse-by-verse-text/'

documents = [] # chapters->verses
for chapter in range(1, 115):
    files = sorted(glob.glob(data_folder + str(chapter).zfill(3) + '*'))
    verses = [open(f, 'r').read() for f in files]
    documents.append(verses)

In [43]:
# Remove non ascii and change to lower case; not necessary with RegexpTokenizer(r'\w+')
docs = [jj.decode('utf8').encode('ascii', errors='ignore').lower() for ii in documents for jj in ii]

In [44]:
# removing whitespaces, punctuations, stopwords, and stemming words
# https://gist.github.com/sangheestyle/8691435
processed = []
tokenizer = RegexpTokenizer(r'\w+')
stop = stopwords.words('english')

# Remove god and lord
stop += ['god', 'lord']

for document in docs:
    intermediate = tokenizer.tokenize(document)    
    intermediate = [i for i in intermediate if i not in stop]
    # FIXME: using other stemmers also to know quality of each stemmed text
    lanste = LancasterStemmer()
    intermediate = [lanste.stem(i) for i in intermediate]
    processed.append(intermediate)

In [10]:
# making dictionary and corpus
dictionary = corpora.Dictionary(processed)
#dictionary.save('/tmp/dict.dict')
corpus = [dictionary.doc2bow(text) for text in processed]
#corpora.MmCorpus.serialize('/tmp/temp.mm', corpus)

In [11]:
# LDA modeling
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=11, passes=10)
corpus_lda = lda[corpus] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [12]:
a = lda.print_topics(num_topics=-1, num_words=7)

In [25]:
docs[:10]

['in the name of god, the gracious, the merciful.',
 'praise be to god, lord of the worlds.',
 'the most gracious, the most merciful.',
 'the path of those you have blessed, not of those against whom there is anger, nor of those who are misguided.',
 'guide us to the straight path.',
 'master of the day of judgment.',
 'it is you we worship, and upon you we call for help.',
 'it is decreed for you: when death approaches one of you, and he leaves wealth, to make a testament in favor of the parents and the relatives, fairly and correctlya duty upon the righteous. ',
 'the sacred month for the sacred month; and sacrilege calls for retaliation. whoever commits aggression against you, retaliate against him in the same measure as he has committed against you. and be conscious of god, and know that god is with the righteous.',
 'thus we made you a moderate community, that you may be witnesses to humanity, and that the messenger may be a witness to you. we only established the direction of pra

In [22]:
print processed[:10]

[['nam', u'gracy', u'mercy'], ['pra', 'world'], [u'gracy', u'mercy'], ['path', 'bless', 'ang', 'misguid'], ['guid', 'us', 'straight', 'path'], ['mast', 'day', 'judg'], ['wor', 'upon', 'cal', 'help'], ['decree', 'dea', 'approach', 'on', 'leav', 'weal', 'mak', 'testa', 'fav', 'par', 'rel', 'fair', 'correctly', 'duty', 'upon', 'right'], ['sacr', 'mon', 'sacr', 'mon', 'sacrileg', 'cal', u'reta', 'whoev', 'commit', u'aggress', u'reta', 'meas', 'commit', u'conscy', 'know', 'right'], ['thu', 'mad', 'mod', 'commun', 'may', 'wit', 'hum', 'messeng', 'may', 'wit', 'est', 'direct', 'pray', 'follow', 'may', u'distinct', 'follow', 'messeng', 'turn', 'heel', 'indee', 'difficult', u'exceiv', 'guid', 'would', 'nev', 'let', 'fai', 'go', 'wast', 'kind', 'toward', 'peopl', u'mercy']]


In [17]:
print len(corpus), len(corpus_lda)

6348 6348


In [13]:
i = 0
print docs[i]
sorted_rel = sorted(corpus_lda[i], key=lambda x: x[1], reverse=True)
print sorted_rel[:3]
print a[sorted_rel[0][0]]
i += 1

in the name of god, the gracious, the merciful.
[(8, 0.7727271975694745), (10, 0.022727347884186636), (5, 0.022727272727404053)]
(8, u'0.113*"mercy" + 0.090*"nam" + 0.082*"gracy" + 0.036*"cre" + 0.017*"say" + 0.015*"hum" + 0.014*"test"')


In [14]:
a

[(0,
  u'0.061*"pun" + 0.022*"upon" + 0.020*"day" + 0.019*"wom" + 0.015*"cal" + 0.014*"around" + 0.013*"pain"'),
 (1,
  u'0.069*"said" + 0.035*"peopl" + 0.023*"among" + 0.022*"wor" + 0.022*"messeng" + 0.021*"us" + 0.016*"say"'),
 (2,
  u'0.070*"know" + 0.039*"heart" + 0.024*"on" + 0.022*"anoth" + 0.021*"rev" + 0.018*"day" + 0.015*"cont"'),
 (3,
  u'0.024*"serv" + 0.022*"us" + 0.019*"believ" + 0.018*"return" + 0.017*"day" + 0.016*"say" + 0.016*"pur"'),
 (4,
  u'0.032*"guid" + 0.027*"say" + 0.026*"turn" + 0.024*"away" + 0.019*"see" + 0.017*"would" + 0.015*"warn"'),
 (5,
  u'0.052*"fir" + 0.021*"ent" + 0.021*"hel" + 0.017*"whoev" + 0.015*"therein" + 0.012*"crush" + 0.011*"hom"'),
 (6,
  u'0.075*"ear" + 0.041*"heav" + 0.038*"say" + 0.035*"day" + 0.028*"everyth" + 0.025*"prom" + 0.019*"belong"'),
 (7,
  u'0.030*"deny" + 0.028*"peopl" + 0.024*"night" + 0.022*"said" + 0.019*"us" + 0.017*"childr" + 0.014*"leav"'),
 (8,
  u'0.113*"mercy" + 0.090*"nam" + 0.082*"gracy" + 0.036*"cre" + 0.017*"say"